In [1]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Set up Selenium options and driver
options = Options()
options.add_argument("--disable-gpu")
# options.add_argument("--headless")  # Optional: Run in headless mode

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Navigate to the page
base_url = "https://www.nykaa.com/brands/chemist-at-play/c/21551?"
params = "ptype=lst&id=21551&root=brand_menu,brand_list,Chemist%20at%20Play"
paginationNumber = 1

# Allow initial time for the page to load
time.sleep(5)

# Scroll incrementally until all products are loaded
scroll_pause_time = 2  # Wait time after each scroll

# Lists to store data
productLinks = []
products = []

while True:

    if paginationNumber == 1:
        url = f"{base_url}{params}"
    else:
        url = f"{base_url}page_no={paginationNumber}&sort=popularity&ptype=lst&id=21551&root=brand_menu,brand_list,Chemist%20at%20Play&eq=desktop"
    
    # Load the Page
    driver.get(url)
    print(f"Scraping Page {paginationNumber}: {url}")
    time.sleep(3)
    
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

    # Wait until the products are loaded
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.productWrapper.css-17nge1h'))
        )
    except TimeoutException:
        print(f"Scraped the Total number of pages: {paginationNumber - 1}")
        break

    # Parse the page content with BeautifulSoup
    content = driver.page_source
    soup = BeautifulSoup(content, 'html.parser')
    
    # Find all product items
    deygaOrganicProducts = soup.findAll('div', class_='productWrapper css-17nge1h')
    if not deygaOrganicProducts:
        print("No Products found on this page.")
        break

    # Extract product links
    for product in deygaOrganicProducts:
        # Extract the Stock Status
        stockTag = product.find('span', class_ = "css-lg5xc9")
        if stockTag:
            stock = stockTag.get_text(strip = True)
        else:
            stock = "In Stock"

        # Extract the Product Highlight
        highlightTag = product.find("li", class_ = "custom-tag css-1bse542")
        if highlightTag:
            highlight = highlightTag.get_text(strip = True)
        else:
            highlight = "None"

        # Extract product link
        linkTag = product.find('a', href=True)
        if linkTag:
            productLink = linkTag['href']
            # Check if the link is a relative URL and add the base URL if necessary
            if not productLink.startswith("http"):  # Relative URL
                productLink = "https://www.nykaa.com" + productLink
            productLinks.append({"link" : productLink, "stock" : stock, "highlight" : highlight})
                
    for productData in productLinks:
        
        link = productData["link"]
        stock = productData["stock"]
        highlight = productData["highlight"]
        
        driver.get(link)
        time.sleep(3)  # Allow time for the page to load

        # Parse individual product page
        productPageContent = driver.page_source
        productSoup = BeautifulSoup(productPageContent, 'html.parser')
    
        # Extract Product Name
        productTag = productSoup.find('h1', class_='css-1gc4x7i')
        if productTag:
            prodName = productTag.get_text(strip=True)
        else:
            prodName = "No Description Available"
    
        # Extract Ratings
        ratingTag = productSoup.find('div', class_="css-1m0y15j")
        if ratingTag:
            ratingText = ratingTag.find('div', class_="css-m6n3ou")
            if ratingText:
                ratings = ratingText.get_text(strip = True)
            else:
                ratings = "N/A"
        else:
            ratings = "N/A"

        # Extract the Ratings & Reviews Count
        ratingCountTag = productSoup.find('div', class_ = "css-1eip5u4")
        if ratingCountTag:
            ratingCount = ratingCountTag.get_text(strip = True)
        else:
            ratingCount = "None"

        # Extract the Offer Price and Original Price
        productDiv = productSoup.find('div', class_ = "css-1d0jf8e")

        # Extract the first two (span) tags
        if productDiv:
            spanTags = productDiv.find_all('span')
            # print(len(spanTags))

            originalPrice = "N/A"
            offerPrice = "N/A"

            if len(spanTags) == 2:
                originalPrice = spanTags[1].get_text(strip = True)
                offerPrice = "N/A"
            elif len(spanTags) == 4:
                originalPrice = spanTags[1].get_text(strip = True)
                offerPrice = spanTags[2].get_text(strip = True)

            # Print the Values
            # print(f"Original Price: {originalPrice}, Offer Price: {offerPrice}")

        else:
            print("Product div not found, defaulting to Unavailable for prices")

        # Extract the Discount
        discountTag = productSoup.find('span', class_ = "css-bhhehx")
        if discountTag:
            discount = discountTag.get_text(strip = True)
        else:
            discount = "No Discount"

        products.append({
            "Product Brand" : "Chemist at Play",
            "Product Name" : prodName,
            "Product Rating" : ratings,
            "Product Rating & Review Count" : ratingCount,
            "Product Original Price" : originalPrice, 
            "Product Offer Price" : offerPrice, 
            "Product Discount" : discount,
            "Product Highlight" : highlight,
            "Product Stock Status" : stock
        })

    # Clear the product links list to avoid the duplication
    productLinks.clear()

    paginationNumber += 1 # Increment the Page Number
    time.sleep(5)

# Close the browser after scraping
driver.quit()

Scraping Page 1: https://www.nykaa.com/brands/chemist-at-play/c/21551?ptype=lst&id=21551&root=brand_menu,brand_list,Chemist%20at%20Play
Scraping Page 2: https://www.nykaa.com/brands/chemist-at-play/c/21551?page_no=2&sort=popularity&ptype=lst&id=21551&root=brand_menu,brand_list,Chemist%20at%20Play&eq=desktop
Scraping Page 3: https://www.nykaa.com/brands/chemist-at-play/c/21551?page_no=3&sort=popularity&ptype=lst&id=21551&root=brand_menu,brand_list,Chemist%20at%20Play&eq=desktop
Scraping Page 4: https://www.nykaa.com/brands/chemist-at-play/c/21551?page_no=4&sort=popularity&ptype=lst&id=21551&root=brand_menu,brand_list,Chemist%20at%20Play&eq=desktop
Scraped the Total number of pages: 3


In [3]:
print(f"Products: \n {products}")

Products: 
 [{'Product Brand': 'Chemist at Play', 'Product Name': 'Chemist At Play Lactic Acid Exfoliating Body Wash (236)& AHA Underarm Roll OnShower gel & Deo Combo(2 pcs)', 'Product Rating': '4.5/5', 'Product Rating & Review Count': '34046ratings&3056reviews', 'Product Original Price': '₹798', 'Product Offer Price': '₹629', 'Product Discount': '21% Off', 'Product Highlight': 'BESTSELLER', 'Product Stock Status': 'In Stock'}, {'Product Brand': 'Chemist at Play', 'Product Name': 'Chemist at Play 5% AHA Underarm Roll On, Lactic Acid Deodrants for Men & Women - Aqua Fragrance(40ml)', 'Product Rating': '4.5/5', 'Product Rating & Review Count': '22661ratings&2153reviews', 'Product Original Price': '₹399', 'Product Offer Price': '₹289', 'Product Discount': '28% Off', 'Product Highlight': 'BESTSELLER', 'Product Stock Status': 'In Stock'}, {'Product Brand': 'Chemist at Play', 'Product Name': 'Chemist At Play Cherry Lip Jelly Lip Balm With SPF 50 PA++++ For Dry, Chapped & Pigmented Lips(10g)'

In [4]:
chemistatPlay  = pd.DataFrame(products)
chemistatPlay

,Product Brand,Product Name,Product Rating,Product Rating & Review Count,Product Original Price,Product Offer Price,Product Discount,Product Highlight,Product Stock Status
0,Chemist at Play,Chemist At Play Lactic Acid Exfoliating Body W...,4.5/5,34046ratings&3056reviews,₹798,₹629,21% Off,BESTSELLER,In Stock
1,Chemist at Play,"Chemist at Play 5% AHA Underarm Roll On, Lacti...",4.5/5,22661ratings&2153reviews,₹399,₹289,28% Off,BESTSELLER,In Stock
2,Chemist at Play,Chemist At Play Cherry Lip Jelly Lip Balm With...,4.8/5,54ratings&39reviews,₹349,₹319,9% Off,None,In Stock
3,Chemist at Play,Chemist At Play Exfoliating Body Wash Shower G...,4.5/5,18004ratings&1523reviews,₹799,₹649,19% Off,BESTSELLER,In Stock
4,Chemist at Play,Chemist At Play 3% AHA Gentle Exfoliating Face...,4.6/5,815ratings&284reviews,₹349,₹329,6% Off,BESTSELLER,In Stock
5,Chemist at Play,Chemist at Play 2% Salicylic Acid Face Wash Fo...,4.4/5,1185ratings&235reviews,₹299,₹269,10% Off,None,In Stock
6,Chemist at Play,Chemist At Play Brightening Body Wash - 3% Vit...,4.5/5,1427ratings&299reviews,₹349,₹329,6% Off,BESTSELLER,In Stock
7,Chemist at Play,Chemist at Play Exfoliating Body Scrub For Rem...,4.5/5,2079ratings&277reviews,₹349,₹319,9% Off,BESTSELLER,In Stock
8,Chemist at Play,Chemist At Play AHA Body Lotion With Niacinami...,4.4/5,3266ratings&642reviews,₹399,₹369,8% Off,BESTSELLER,In Stock
9,Chemist at Play,Chemist at Play Foot Cream for Cracked Heels &...,4.6/5,457ratings&94reviews,₹349,₹319,9% Off,None,In Stock


In [5]:
# chemistatPlay.to_csv("C:/Users/Logiya Vidhyapathy/Documents/KGISL Data Science/Project Files and Documents/Capstone Project/Chemist at Play Products_Nykaa.csv", index = False)